In [14]:
import sqlite3 as sq
import pandas as pd

Создание соединения с базой данных

In [15]:
with sq.connect('data/checking-logs.sqlite') as conn:
    conn.cursor()

Создай новую таблицу datamart в базе данных.

In [16]:
datamart = pd.io.sql.read_sql('''SELECT c.uid, c.labname, c.timestamp as first_commit_ts,
                                MIN(p.datetime) as first_view_ts
                                FROM checker as c
                                LEFT JOIN pageviews as p
                                ON c.uid = p.uid
                                WHERE c.status = "ready"
                                      AND c.numTrials = 1
                                      AND c.labname IN ("laba04", "laba04s", "laba05", "laba06", "laba06s", "project1")
                                      AND c.uid LIKE "user_%"
                                GROUP BY c.timestamp, c.uid''',
                              conn,
                              parse_dates=['first_commit_ts', 'first_view_ts'])

# Сохранение результатов в таблицу datamart в базе данных
datamart.to_sql('datamart', conn, if_exists='replace')
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


Используя методы библиотеки Pandas, создай два датафрейма: test и control

In [17]:
# Создание датафрейма test, содержащего пользователей с непустыми значениями first_view_ts
test = datamart[datamart['first_view_ts'].notnull()]

# Создание датафрейма control, содержащего пользователей с пустыми значениями first_view_ts
control = datamart[datamart['first_view_ts'].isnull()]

# Замена пропущенных значений в control средним значением first_view_ts из test
mean_f = test['first_view_ts'].mean()
control['first_view_ts'] = control['first_view_ts'].fillna(mean_f)

# Сохранение датафреймов test и control в базе данных
test.to_sql('test', conn, if_exists='replace')
control.to_sql('control', conn, if_exists='replace')

C:\Users\1\AppData\Local\Temp\ipykernel_13872\990322478.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control['first_view_ts'] = control['first_view_ts'].fillna(mean_f)


81

Закрытие соединения с базой данных

In [18]:
conn.close()

# Чек

In [19]:
with sq.connect('data/checking-logs.sqlite') as conn:
    conn.cursor()

In [20]:
datamart.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 140 entries, 0 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              140 non-null    object        
 1   labname          140 non-null    object        
 2   first_commit_ts  140 non-null    datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 4.5+ KB


In [21]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59 entries, 3 to 139
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              59 non-null     object        
 1   labname          59 non-null     object        
 2   first_commit_ts  59 non-null     datetime64[ns]
 3   first_view_ts    59 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 2.3+ KB


In [22]:
control.info()

<class 'pandas.core.frame.DataFrame'>
Index: 81 entries, 0 to 137
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   uid              81 non-null     object        
 1   labname          81 non-null     object        
 2   first_commit_ts  81 non-null     datetime64[ns]
 3   first_view_ts    81 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 3.2+ KB


In [23]:
test = pd.io.sql.read_sql('SELECT *FROM test', conn)


In [24]:
control = pd.io.sql.read_sql('SELECT *FROM control', conn)

In [25]:
conn.close()